In [0]:
# Import 
from pyspark.sql.functions import *


# List of paths for the large monthly trip data files (Parquet).
TRIP_DATA_PATHS = [
    f"/Volumes/nyc_yellow_tripdata/default/filestore/yellow_tripdata_2025-{month:02d}.parquet" 
    for month in range(1, 9) # This generates months 01-08
]

# Path for the Taxi Zone Lookup CSV
LOOKUP_TABLE_PATH = "/Volumes/nyc_yellow_tripdata/default/filestore/taxi_zone_lookup.csv"


# --- Ingestion: Read and Union Trip Data ---

print("Starting ingestion and union of 8 months of trip data...")


raw_trips_df = (spark.read
                .format("parquet")
                .load(TRIP_DATA_PATHS)
               )

# Verifying the load by counting and displaying a sample
total_records = raw_trips_df.count()
print(f"Total raw records loaded: {total_records}")

print("\nRaw Data Schema:")
# Print the data types to ensure they were inferred correctly
raw_trips_df.printSchema()

print("\nRaw Data Sample:")
raw_trips_df.limit(5).display()

In [0]:

print("Starting initial cleaning and filtering...")

# Filtering out records with invalid or impossible values ensures the integrity of all subsequent analytical calculations (e.g., averages, sums, and ML model training)

cleaned_trips_df = (raw_trips_df
    # TRIP DISTANCE FILTER
    # A trip distance of 0 is either an error or a canceled ride where the meter was not reset.
    # I am exclduing these records from the dataset as they create infinite speed values (distance/time) and skew metrics.
    .filter(col("trip_distance") > 0)
    
    # FARE AMOUNT FILTER
    # A fare of less than $1 suggests a free trip, a severe error, or a test transaction.
    .filter(col("fare_amount") >= 1.0)
    
    # PASSENGER COUNT FILTER
    # A passenger count of 0 is illogical for a completed trip and usually indicates a data entry error.
    .filter(col("passenger_count") > 0)
    
    # 4. PAYMENT TYPE FILTER
    # Focusing on payment types 1 (Credit Card) and 2 (Cash) provides the most reliable data
    # for revenue analysis. Types 3 (No Charge) and 4 (Dispute) are typically excluded from core revenue KPIs.
    .filter(col("payment_type").isin([1, 2])) 
    
    # COLUMN RENAMING
    # Renaming columns to a cleaner, more generalized standard to make the code easier to read
    # and maintain, as 'tpep_' prefixes are specific to "Yellow Taxi" data.
    .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
    .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
)

# Check removed rows
initial_records = raw_trips_df.count()
final_records = cleaned_trips_df.count()
print(f"Initial Records: {initial_records}")
print(f"Records after cleaning: {final_records}")
print(f"Records removed: {initial_records - final_records}")

# Display schema 
print("\nCleaned Data Schema:")
cleaned_trips_df.printSchema()

In [0]:
print("\nStarting feature engineering...")


transformed_df = (cleaned_trips_df
    # TRIP DURATION CALCULATION
    # Feature: 'trip_duration_seconds'
    # This feature is essential for time-based analysis (e.g., identifying high-traffic hours) and is necessary to calculate speed. PySpark `unix_timestamp` function for fast, distributed calculation.
    .withColumn(
        "trip_duration_seconds",
        unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))
    )
    
    # AVERAGE SPEED CALCULATION
    # Feature: 'avg_speed_mph'
    # Average speed is a powerful metric for analyzing city congestion, driver efficiency, and identifying potential outliers (like trips with impossible speeds).
    .withColumn(
        "avg_speed_mph",
        # Formula: distance / (duration_in_seconds / 3600 seconds per hour)
        col("trip_distance") / (col("trip_duration_seconds") / 3600)
    )
    
    #  FINAL DURATION FILTER
    # A trip lasting less than 60 seconds is physically unlikely for the NYC system. 
    # This filter removes more noise, ensuring we focus on legitimate completed trips.
    .filter(col("trip_duration_seconds") > 60) 
    
    # FINAL SPEED FILTER
    # Speeds over 60 MPH are nearly impossible within city limits. This acts as a final layer of quality control, identifying and removing trips that are likely GPS errors or data corruption.
    .filter(col("avg_speed_mph") <= 60) 
    
    # SELECT AND ORDER COLUMNS
    # Explicitly selecting columns ensures only relevant fields are carried forward, improving performance (by reducing data size) and providing a clean output schema for the next phase.
    .select(
        "VendorID", "pickup_datetime", "dropoff_datetime", 
        "PULocationID", "DOLocationID", "passenger_count",
        "trip_distance", "fare_amount", "tip_amount", 
        "total_amount", "payment_type", 
        "trip_duration_seconds", "avg_speed_mph"
    )
)

# Final verification of the total count after all filters
print(f"Final records after feature engineering and final speed filter: {transformed_df.count()}")
transformed_df.limit(5).display()

In [0]:
print("Starting ingestion of the small lookup table...")


# Read the CSV file (lookup table witj zip codes)
lookup_df = (spark.read
             .format("csv")
             .option("header", "true") #The first row is the header
             .option("inferSchema", "true") # guess the correct data types
             .load(LOOKUP_TABLE_PATH)
            )

print("\nLookup Table Schema:")
lookup_df.printSchema()

print("\nLookup Table Sample:")
lookup_df.limit(5).display()

In [0]:

print("\nJoining trip data with the lookup table...")

# This step creates the final, ready-for-analysis dataset.

# Create the final joined DataFrame
joined_df = (transformed_df
    # Join the main trip data with the lookup table on the Pickup Location ID (PULocationID)
    .join(
        lookup_df, 
        # Define the join condition: Trip's pickup ID must match the Lookup table's ID
        (transformed_df["PULocationID"] == lookup_df["LocationID"]), 
        "inner" # Inner join to ensure every trip has a valid, known zone name
    )
    .select(
        transformed_df["*"], # Keep all columns from the trip data
        lookup_df["Zone"].alias("Pickup_Zone"),
        lookup_df["Borough"].alias("Pickup_Borough")
    )
    # Drop the original PULocationID column as the zone name is more useful now
    .drop("PULocationID")
)

# Display a sample of the joined data 
print(f"Total records after join: {joined_df.count()}")
print("\nJoined Data Sample:")
joined_df.limit(5).display()

In [0]:
print("\nStarting calculation of 7-day Rolling Average Fare...")

# --- Feature Engineering: Create Date Columns ---
# Extracting the date (year, month, day) from the timestampfor grouping, partitioning,and ordering the data within the Window Function.
dated_df = (joined_df
    .withColumn("pickup_date", to_date(col("pickup_datetime")))
    .withColumn("pickup_year", year(col("pickup_datetime")))
    .withColumn("pickup_month", month(col("pickup_datetime")))
)

# --- Define the Window Specification ---
# --- Partions ---
# Partition By: 'Pickup_Zone' to ensure the rolling average calculation restarts for every zone.
# Order By: 'pickup_date' to ensure the calculation is applied chronologically.
# Rows Between: '-7' (7 days before) and '0' (the current day) defines the 7-day rolling window.
from pyspark.sql.window import Window

zone_daily_window = (Window
    .partitionBy("Pickup_Zone")
    .orderBy("pickup_date")
    .rowsBetween(-7, 0) 
)

# --- Apply the Window Function ---
rolling_avg_df = (dated_df
    .withColumn(
        "7_day_rolling_avg_fare",
        # Calculate the average fare amount over the defined 7-day window
        avg("fare_amount").over(zone_daily_window)
    )
    .withColumn(
        "7_day_rolling_avg_speed",
        # Calculate the average speed over the same window
        avg("avg_speed_mph").over(zone_daily_window)
    )
    .orderBy("Pickup_Zone", "pickup_date") # Order for clear visualization
)

print("\nRolling Average Calculation Complete. Sample data:")
rolling_avg_df.select(
    "Pickup_Zone", "pickup_date", "fare_amount", "7_day_rolling_avg_fare", "7_day_rolling_avg_speed"
).limit(10).display()

In [0]:
print("\nStarting Daily Revenue and Trip Count Aggregation...")

# --- Group and Aggregate ---

daily_summary_df = (rolling_avg_df
    .groupBy("pickup_date", "Pickup_Borough") # Group by date and borough to see daily performance per area
    .agg(
        # Calculate the total fare revenue for the day
        sum("fare_amount").alias("Total_Daily_Revenue"),
        # Count the total number of trips
        count(lit(1)).alias("Total_Trips"),
        # Calculate the overall average speed for the day
        avg("avg_speed_mph").alias("Overall_Daily_Avg_Speed")
    )
    .orderBy("pickup_date", "Pickup_Borough")
)

print("\nDaily Summary Aggregation Complete. Sample data:")
daily_summary_df.limit(10).display()

In [0]:
# --- Initial Load (L) to Delta Lake ---
TARGET_TABLE_NAME = "nyc_taxi_analysis_baseline"

print(f"\nStarting Initial Load to Delta Table: {TARGET_TABLE_NAME}...")

# Partitioning by Year/Month to optimize data skipping during future queries.

(rolling_avg_df
    .write
    .mode("overwrite") # Overwrite the table each time the notebook runs
    .format("delta")
    #  Partitioning by Year and Month to optimize data skipping, to make queries much faster.
    .partitionBy("pickup_year", "pickup_month") 
    .saveAsTable(TARGET_TABLE_NAME) # Save the data as a managed table in Unity Catalog
)

print(f"Initial Load Complete. Baseline table '{TARGET_TABLE_NAME}' created and partitioned.")

In [0]:

#  --- Broadcast Join ---
# Force Spark to send the entire small table to every worker node, eliminating the need 
# to shuffle the massive trip data, which will drastically reduces network overhead and execution time.

from pyspark.sql.functions import broadcast

print("\nStarting Optimized Join (Broadcast Join)...")

# --- Optimized Join ---
optimized_joined_df = (transformed_df
    # Apply the broadcast hint to the smaller DataFrame (lookup_df)
    .join(
        broadcast(lookup_df), 
        (transformed_df["PULocationID"] == lookup_df["LocationID"]), 
        "inner"
    )
    # Select and rename columns, identical to the previous join step
    .select(
        transformed_df["*"],
        lookup_df["Zone"].alias("Pickup_Zone_Optimized"), # Rename for comparison
        lookup_df["Borough"].alias("Pickup_Borough_Optimized")
    )
    .drop("PULocationID")
)

optimized_joined_df.limit(5).display()

In [0]:
# This re-uses the Window definition calculated above
from pyspark.sql.window import Window
zone_daily_window = (Window.partitionBy("Pickup_Zone_Optimized").orderBy("pickup_date").rowsBetween(-7, 0))

optimized_final_df = (optimized_joined_df
    .withColumn("pickup_date", to_date(col("pickup_datetime")))
    .withColumn("pickup_year", year(col("pickup_datetime")))
    .withColumn("pickup_month", month(col("pickup_datetime")))
    .withColumn(
        "7_day_rolling_avg_fare_opt",
        avg("fare_amount").over(zone_daily_window)
    )
)

# --- Final Load (L) to Delta Lake - OPTIMIZED ---

TARGET_TABLE_NAME_OPT = "nyc_taxi_analysis_optimized"

print(f"\nStarting Final Load to Optimized Delta Table: {TARGET_TABLE_NAME_OPT}...")

# Write the fully optimized pipeline output
(optimized_final_df
    .write
    .mode("overwrite")
    .format("delta")
    .partitionBy("pickup_year", "pickup_month") 
    .saveAsTable(TARGET_TABLE_NAME_OPT)
)

print(f"Optimized Load Complete. Table '{TARGET_TABLE_NAME_OPT}' created.")